# Data Reading Section:
##### In this section, we will use the following codes, which will be explained below, to read the data we need from the main reference file and convert it to CSV format so that we can use it in the next steps.

---

# ⚠️ IMPORTANT — READ BEFORE RUNNING

This notebook expects the raw dataset to be available **before execution**.  
If the required ZIP file is not placed in the correct path, the ETL pipeline will fail or generate incomplete/duplicated outputs.

---

## ✅ Required Action

Please make sure the following file exists **before running the notebook**:

..\data\raw\tennis_data.zip


> 📌 The path is already configured inside the notebook’s Python extraction script — do **not** change it unless necessary.

If the ZIP file has a different name, please rename it or update the code accordingly.

---

# Part 1: Importing the required libraries, defining the paths, and creating the required directories if they do not exist.
In this section, we import the libraries and items we need to use them later, and then we define the main paths, such as the main zip file path, the output file directory, and the temp directory, in a relational manner, to be included in the data folder of this project.

In [ ]:
import os, zipfile
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from io import BytesIO
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

print(os.getcwd())


# Define paths
main_zip = r".\data\raw\tennis_data.zip"
output_dir = r".\data\processed"
temp_dir = r".\data\raw\temp"
base_path = r"..\data\processed"
clean_path = r"..\data\processed\clean"
etl_files_path = r"..\data\processed\clean"

os.makedirs(clean_path, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(temp_dir, exist_ok=True)

# Part 2: Creating a CSV table generator function from Parquet files
In this section, we have created a very useful function that, based on the keyword of the parquet category name that we give it, goes to the defined path of the main zip file and reads the parquets belonging to the specified tables and the data related to the specified columns from the zip files for each day. In addition to all this, we specify that the records of this table should be unique based on the unique data identifier or that this table can have multiple rows for each unique identifier. Our unique identifier is match_id.

In [ ]:
def build_table(table_keyword, needed_cols, output_name, dedup_on="match_id"):
    """
    table_keyword: like 'event_' or 'home_team_'
    needed_cols: list of needed columns
    output_name: name of output CSV file
    dedup_on: unique column for deduplication (default is 'match_id')
    """
    csv_path = os.path.join(output_dir, output_name)
    if os.path.exists(csv_path):
        os.remove(csv_path)

    all_dfs = []
    row_counter = 0

    with zipfile.ZipFile(main_zip, "r") as main_zip_ref:
        daily_zips = main_zip_ref.namelist()
        print(f"📦 Count of daily zips: {len(daily_zips)}")

        for i, daily_zip_name in enumerate(daily_zips, start=1):
            print(f"🔹 ({i}/{len(daily_zips)}) processing {daily_zip_name} ...")
            main_zip_ref.extract(daily_zip_name, temp_dir)
            daily_zip_path = os.path.join(temp_dir, daily_zip_name)

            with zipfile.ZipFile(daily_zip_path, "r") as daily_zip_ref:
                parquet_files = [f for f in daily_zip_ref.namelist() if f.endswith(".parquet") and table_keyword in f]
                for f in parquet_files:
                    with daily_zip_ref.open(f) as pf:
                        table = pq.read_table(BytesIO(pf.read()))
                        df = table.to_pandas()
                        df = df[[c for c in needed_cols if c in df.columns]]
                        df["date_source"] = daily_zip_name.replace(".zip", "")
                        all_dfs.append(df)
                        row_counter += len(df)

            os.remove(daily_zip_path)

    if all_dfs:
        df_all = pd.concat(all_dfs, ignore_index=True)
        print(f"✅ Shape: {df_all.shape}")
        if dedup_on and dedup_on in df_all.columns:
            df_all = df_all.drop_duplicates(subset=dedup_on)
        else:
            df_all = df_all.drop_duplicates()
        print(f"🧹 after cleaning duplicated rows: {df_all.shape}")

        df_all.to_csv(csv_path, index=False)
        print(f"💾 Saved: {csv_path}")
        print(f"📊 Count of all rows: {len(df_all)}")
    else:
        print(f"⚠️ There is no file for {table_keyword}")

# Part 3: Using the above cell function and creating CSVs of the tables required for analysis according to the columns required from them
In this part, based on the initial analysis we had of the 17 questions in question and the data they required, we extracted a series of tables from a total of 15 tables and a series of their columns that were needed to analyze and answer the 17 questions we needed. Here, we want to extract them from the original raw zip file and convert them to CSV files so that we can use these files later in analyzing and answering the questions.

In [ ]:
build_table(
    table_keyword="event_",
    needed_cols=["match_id", "first_to_serve", "winner_code", "default_period_count", "start_datetime", "match_slug"],
    output_name="event.csv",
    dedup_on="match_id"
)

build_table(
    table_keyword="home_team_",
    needed_cols=["match_id", "player_id", "full_name", "gender", "height", "weight", "plays", "current_rank", "country"],
    output_name="home_team.csv",
    dedup_on="match_id"
)

build_table(
    table_keyword="away_team_",
    needed_cols=["match_id", "player_id", "full_name", "gender", "height", "weight", "plays", "current_rank", "country"],
    output_name="away_team.csv",
    dedup_on="match_id"
)

build_table(
    table_keyword="tournament_",
    needed_cols=["match_id", "tournament_id", "tournament_name", "ground_type", "tennis_points", "start_datetime"],
    output_name="tournament.csv",
    dedup_on="match_id"
)

build_table(
    table_keyword="time_",
    needed_cols=["match_id", "period_1", "period_2", "period_3", "period_4", "period_5", "current_period_start_timestamp"],
    output_name="time.csv",
    dedup_on="match_id"
)

build_table(
    table_keyword="statistics_",
    needed_cols=["match_id", "statistic_name", "home_value", "away_value"],
    output_name="statistics.csv",
    dedup_on=None  # No deduplication because we have multiple rows per match_id in statistics
)

build_table(
    table_keyword="power_",
    needed_cols=["match_id", "set_num", "game_num", "value", "break_occurred"],
    output_name="power.csv",
    dedup_on=None # No deduplication because we have multiple rows per match_id in power
)

build_table(
    table_keyword="pbp_",
    needed_cols=["match_id", "set_id", "game_id", "point_id", "home_point", "away_point", "home_score"],
    output_name="pbp.csv",
    dedup_on=None # No deduplication because we have multiple rows per match_id in pbp
)


##  Part 4: Data Cleaning Stage
#### In this section, we will clean the extracted CSV files created in the previous section.
# 
### **Goal:**  
- Remove duplicate rows  
 - Handle missing values (`NaN`)  
 - Standardize data types  
 The cleaned outputs will be stored in `../data/clean` for the next normalization phase.


###  Cleaning: Event Table

In [ ]:
df_event = pd.read_csv(os.path.join(base_path, "event.csv"))
df_event.drop_duplicates(inplace=True)

for col in df_event.columns:
    if df_event[col].dtype == 'object':
        df_event[col] = df_event[col].fillna("Unknown")
    else:
        df_event[col] = df_event[col].fillna(0)

if "match_id" in df_event.columns:
    df_event["match_id"] = df_event["match_id"].astype(str)

df_event.to_csv(os.path.join(clean_path, "event_clean.csv"), index=False)
print("✅ event_clean.csv created successfully!")

###  Cleaning: Home Team Table

In [ ]:
df_home = pd.read_csv(os.path.join(base_path, "home_team.csv"))
df_home = df_home.drop_duplicates()

string_cols = ["full_name", "gender", "plays", "country"]
numeric_cols = ["height", "weight", "current_rank"]

for col in string_cols:
    if col in df_home.columns:
        df_home[col] = df_home[col].fillna("Unknown")

for col in numeric_cols:
    if col in df_home.columns:
        df_home[col] = df_home[col].fillna(0)

if "match_id" in df_home.columns:
    df_home["match_id"] = df_home["match_id"].astype(str)

df_home.to_csv(os.path.join(clean_path, "home_team_clean.csv"), index=False)
print("✅ home_team_clean.csv created successfully!")

###  Cleaning: Away Team Table

In [ ]:
df_away = pd.read_csv(os.path.join(base_path, "away_team.csv"))
df_away = df_away.drop_duplicates()

string_cols = ["full_name", "gender", "plays", "country"]
numeric_cols = ["height", "weight", "current_rank"]

for col in string_cols:
    if col in df_away.columns:
        df_away[col] = df_away[col].fillna("Unknown")

for col in numeric_cols:
    if col in df_away.columns:
        df_away[col] = df_away[col].fillna(0)

if "match_id" in df_away.columns:
    df_away["match_id"] = df_away["match_id"].astype(str)

df_away.to_csv(os.path.join(clean_path, "away_team_clean.csv"), index=False)
print("✅ away_team_clean.csv created successfully!")

### Cleaning: Tournamet Table

In [ ]:
df_tournament = pd.read_csv(os.path.join(base_path, "tournament.csv"))
df_tournament = df_tournament.drop_duplicates()

mask = df_tournament['ground_type'].isnull() | (df_tournament['ground_type'].str.strip() == '')
df_tournament.loc[mask, 'ground_type'] = 'Unknown'

df_tournament.to_csv(os.path.join(clean_path, "tournament_clean.csv"), index=False)
print("✅ tournament_clean.csv created successfully!")

### Cleaning: Statistics Table

In [ ]:
df_statistics = pd.read_csv(os.path.join(base_path, "statistics.csv"))
df_statistics = df_statistics.drop_duplicates()

# This data dosent have any nulls to clean however we display the count of nulls for verification
display(df_statistics.isnull().sum())

if "match_id" in df_statistics.columns:
    df_statistics["match_id"] = df_statistics["match_id"].astype(str)

df_statistics.to_csv(os.path.join(clean_path, "statistics_clean.csv"), index=False)
print("✅ statistics_clean.csv created successfully!")

### Cleaning: Time Table

In [ ]:
df_time = pd.read_csv(os.path.join(base_path, "time.csv"))
df_time = df_time.drop_duplicates()

df_time.drop(columns=["period_4", "period_5"], inplace=True) # because all tennis matches are best of 3 sets

if "match_id" in df_time.columns:
    df_time["match_id"] = df_time["match_id"].astype(int)

df_time.to_csv(os.path.join(clean_path, "time_clean.csv"), index=False)
print("✅ time_clean.csv created successfully!")

### Cleaning: Point By Point Table

In [ ]:
df_pbp = pd.read_csv(os.path.join(base_path, "pbp.csv"))
df_pbp = df_pbp.drop_duplicates()

df_pbp['home_point'] = df_pbp['home_point'].replace('A', 1).astype(int)
df_pbp['away_point'] = df_pbp['away_point'].replace('A', 1).astype(int)

# This data dosent have any nulls to clean however we display the count of nulls for verification
display(df_pbp.isnull().sum())

if "match_id" in df_pbp.columns:
    df_pbp["match_id"] = df_pbp["match_id"].astype(str)


df_pbp.to_csv(os.path.join(clean_path, "pbp_clean.csv"), index=False)
print("✅ pbp_clean.csv created successfully!")

### Cleaning: Power Table

In [ ]:
df_power = pd.read_csv(os.path.join(base_path, "power.csv"))
df_power = df_power.drop_duplicates()

# No nulls to clean, just save the cleaned file and other column is clean as you can see here
display("Sum of nulls:", df_power.isnull().sum())
display("Sum of invalid game_num entries:", df_power['match_id'][df_power['game_num'] < 1].sum())
display("DataFrame dtypes for verification correctness of data:", df_power.dtypes)

df_power.to_csv(os.path.join(clean_path, "power_clean.csv"), index=False)
print("✅ power_clean.csv created successfully!")

## Part 5: Normalization Stage
#### Now that we have clean CSVs, in this part we will:
#
 - Convert data types (e.g., timestamps to datetime)  
 - Standardize text (e.g., capitalization, spacing)  
 - Fill remaining missing values intelligently (using mean, median, or mode)  
 The normalized final datasets will be saved in `../data/processed/clean` as `_final.csv` files.


###  Normalization — Event Table

In [ ]:
input_path = os.path.join(clean_path, "event_clean.csv")
output_path = os.path.join(clean_path, "event_final.csv")

df_event = pd.read_csv(input_path)

df_event["match_id"] = df_event["match_id"].astype(int)
df_event["default_period_count"] = df_event["default_period_count"].astype(int)
df_event["date_source"] = df_event["date_source"].astype(int)

if np.issubdtype(df_event["start_datetime"].dtype, np.number):
    df_event["start_datetime"] = pd.to_datetime(df_event["start_datetime"], unit="s", errors="coerce")

df_event["winner_code"] = df_event["winner_code"].fillna(df_event["winner_code"].mode()[0])
df_event["first_to_serve"] = df_event["first_to_serve"].fillna(df_event["first_to_serve"].mode()[0])

df_event.to_csv(output_path, index=False)
print("✅ event_final.csv created successfully!")
print(df_event.info())
print(df_event.isna().sum())

###  Normalization — Home Team Table

In [ ]:
input_path = os.path.join(clean_path, "home_team_clean.csv")
output_path = os.path.join(clean_path, "home_team_final.csv")

df_home = pd.read_csv(input_path)

numeric_cols = ["height", "weight", "current_rank"]
for col in numeric_cols:
    if col in df_home.columns:
        df_home[col] = pd.to_numeric(df_home[col], errors="coerce")

if "gender" in df_home.columns:
    df_home["gender"] = df_home["gender"].astype(str).str.strip().str.title().replace({"Nan":"Unknown"})
if "plays" in df_home.columns:
    df_home["plays"] = df_home["plays"].astype(str).str.strip().str.lower().replace({"nan":"unknown"})
for col in ["full_name", "country"]:
    if col in df_home.columns:
        df_home[col] = df_home[col].astype(str).str.strip()

if "height" in df_home.columns:
    df_home["height"] = df_home["height"].fillna(df_home["height"].mean(skipna=True))
if "weight" in df_home.columns:
    df_home["weight"] = df_home["weight"].fillna(df_home["weight"].mean(skipna=True))
if "current_rank" in df_home.columns:
    df_home["current_rank"] = df_home["current_rank"].fillna(df_home["current_rank"].median(skipna=True))

for col in ["gender", "plays"]:
    if col in df_home.columns:
        mode_val = df_home[col].mode(dropna=True)
        if not mode_val.empty:
            df_home[col] = df_home[col].fillna(mode_val.iloc[0])
        else:
            df_home[col] = df_home[col].fillna("Unknown")

for col in ["player_id", "full_name", "country"]:
    if col in df_home.columns:
        df_home[col] = df_home[col].fillna("Unknown")

if "match_id" in df_home.columns:
    df_home["match_id"] = df_home["match_id"].astype(int)

df_home.to_csv(output_path, index=False)
print("✅ home_team_final.csv created successfully!")
print(df_home.info())
print(df_home.isna().sum())

###  Normalization — Away Team Table

In [ ]:
input_path = os.path.join(clean_path, "away_team_clean.csv")
output_path = os.path.join(clean_path, "away_team_final.csv")

df_away = pd.read_csv(input_path)

numeric_cols = ["height", "weight", "current_rank"]
for col in numeric_cols:
    if col in df_away.columns:
        df_away[col] = pd.to_numeric(df_away[col], errors="coerce")

if "gender" in df_away.columns:
    df_away["gender"] = df_away["gender"].astype(str).str.strip().str.title().replace({"Nan":"Unknown"})
if "plays" in df_away.columns:
    df_away["plays"] = df_away["plays"].astype(str).str.strip().str.lower().replace({"nan":"unknown"})
for col in ["full_name", "country"]:
    if col in df_away.columns:
        df_away[col] = df_away[col].astype(str).str.strip()

if "height" in df_away.columns:
    df_away["height"] = df_away["height"].fillna(df_away["height"].mean(skipna=True))
if "weight" in df_away.columns:
    df_away["weight"] = df_away["weight"].fillna(df_away["weight"].mean(skipna=True))
if "current_rank" in df_away.columns:
    df_away["current_rank"] = df_away["current_rank"].fillna(df_away["current_rank"].median(skipna=True))

for col in ["gender", "plays"]:
    if col in df_away.columns:
        mode_val = df_away[col].mode(dropna=True)
        if not mode_val.empty:
            df_away[col] = df_away[col].fillna(mode_val.iloc[0])
        else:
            df_away[col] = df_away[col].fillna("Unknown")

for col in ["player_id", "full_name", "country"]:
    if col in df_away.columns:
        df_away[col] = df_away[col].fillna("Unknown")

if "match_id" in df_away.columns:
    df_away["match_id"] = df_away["match_id"].astype(int)

df_away.to_csv(output_path, index=False)
print("✅ away_team_final.csv created successfully!")
print(df_away.info())
print(df_away.isna().sum())

###  Normalization — Tournament Table

In [ ]:
input_path = os.path.join(clean_path, "tournament_clean.csv")
output_path = os.path.join(clean_path, "tournament_final.csv")

df_tournament = pd.read_csv(input_path)

df_tournament['match_id'] = df_tournament['match_id'].astype(int)

df_tournament.to_csv(output_path, index=False)
print("✅ tournament_final.csv created successfully!")
print(df_tournament.info())
print(df_tournament.isna().sum())

### Normalization — Statistics Table

In [ ]:
input_path = os.path.join(clean_path, "statistics_clean.csv")
output_path = os.path.join(clean_path, "statistics_final.csv")

df_statistics = pd.read_csv(input_path)

df_statistics['date_source'] = pd.to_datetime(df_statistics['date_source'], format='%Y%m%d')
df_statistics['home_value'] = pd.to_numeric(df_statistics['home_value'], errors='coerce')
df_statistics['away_value'] = pd.to_numeric(df_statistics['away_value'], errors='coerce')
df_statistics['statistic_name'] = df_statistics['statistic_name'].astype(str).str.replace(" ", "_").str.lower()
df_statistics['match_id'] = df_statistics['match_id'].astype(int)



df_statistics.to_csv(output_path, index=False)
print("✅ statistics_final.csv created successfully!")
print(df_statistics.info())
print(df_statistics.isna().sum())

### Normalization — Time Table

In [ ]:
input_path = os.path.join(clean_path, "time_clean.csv")
output_path = os.path.join(clean_path, "time_final.csv")

df_time = pd.read_csv(input_path)

periods = ["period_1", "period_2", "period_3"]

df_time["match_id"] = df_time["match_id"].astype(int)
df_time['date_source'] = pd.to_datetime(df_time['date_source'], format='%Y%m%d')
df_time['current_period_start_timestamp'] = pd.to_datetime(df_time['current_period_start_timestamp'], unit='s', errors='coerce')

df_time['match_id'] = df_time['match_id'].astype(int)

MS_TRESHOLD = 100_000  # 100,000 milliseconds = 100 seconds

for period in periods:
    df_time[period] = pd.to_numeric(df_time[period], errors='coerce').abs()
    mask = df_time[period] > MS_TRESHOLD
    df_time.loc[mask, period] = df_time.loc[mask, period] / 1000 # convert milliseconds to seconds


df_time.to_csv(output_path, index=False)
print("✅ time_final.csv created successfully!")

### Normalization — Point By Point Table

In [ ]:
input_path = os.path.join(clean_path, "pbp_clean.csv")
output_path = os.path.join(clean_path, "pbp_final.csv")

df_pbp = pd.read_csv(input_path)

df_pbp['date_source'] = pd.to_datetime(df_pbp['date_source'], format='%Y%m%d')
df_pbp['match_id'] = df_pbp['match_id'].astype(int)
df_pbp['home_point'] = df_pbp['home_point'].astype(int)
df_pbp['away_point'] = df_pbp['away_point'].astype(int)
df_pbp['home_score'] = df_pbp['home_score'].astype(int)
df_pbp['set_id'] = df_pbp['set_id'].astype(int)
df_pbp['game_id'] = df_pbp['game_id'].astype(int)
df_pbp['point_id'] = df_pbp['point_id'].astype(int)

df_pbp.to_csv(output_path, index=False)
print("✅ pbp_final.csv created successfully!")

### Normalization — Power Table

In [ ]:
input_path = os.path.join(clean_path, "pbp_clean.csv")
output_path = os.path.join(clean_path, "pbp_final.csv")

df_pbp = pd.read_csv(input_path)

df_pbp['date_source'] = pd.to_datetime(df_pbp['date_source'], format='%Y%m%d')
df_pbp['match_id'] = df_pbp['match_id'].astype(int)
df_pbp['home_point'] = df_pbp['home_point'].astype(int)
df_pbp['away_point'] = df_pbp['away_point'].astype(int)
df_pbp['set_id'] = df_pbp['set_id'].astype(int)
df_pbp['game_id'] = df_pbp['game_id'].astype(int)
df_pbp['point_id'] = df_pbp['point_id'].astype(int)

df_pbp.to_csv(output_path, index=False)
print("✅ pbp_final.csv created successfully!")

### Question 1 — How many tennis players are included in the dataset?

To find the number of unique tennis players, we combine the home and away team tables, remove duplicate players based on `player_id`, and count how many unique players remain.


In [ ]:
import pandas as pd
import numpy as np
import os

base = "/Users/macbook/Downloads/Daneshkar/tennis project/TennisProject/data/processed/clean"

# Load cleaned & normalized player data
df_home = pd.read_csv(os.path.join(base, "home_team_final.csv"))
df_away = pd.read_csv(os.path.join(base, "away_team_final.csv"))

# Combine home and away players
players = pd.concat([df_home, df_away], ignore_index=True)

# Keep only unique players
unique_players = players.drop_duplicates(subset="player_id")

# Count unique players
total_unique_players = unique_players.shape[0]

print("total unique players =",total_unique_players)

###  Question 2 — What is the average height of the players?
The goal of this question is to calculate the average height of the tennis players in the dataset.
To do this, we first need to create a unique players table so that players who are duplicates in home and away are not counted again.
Then we correct invalid heights (such as 0 or NaN) and calculate the true average.

In [ ]:
base = "/Users/macbook/Downloads/Daneshkar/tennis project/TennisProject/data/processed/clean"
# load data
df_home = pd.read_csv(os.path.join(base, "home_team_final.csv"))
df_away = pd.read_csv(os.path.join(base, "away_team_final.csv"))

# combine & remove duplicate players
players = pd.concat([df_home , df_away], ignore_index=True)
players_unique = players.drop_duplicates(subset="player_id").copy()

# replace zeros with NaN
players_unique.loc[:, "height"] = players_unique["height"].replace(0, np.nan)

# fill missing heights with mean
mean_height = players_unique["height"].mean(skipna=True)
players_unique.loc[:, "height"] = players_unique["height"].fillna(mean_height)

# Calculate average height
average_height = players_unique["height"].mean()
print("Average height =" , average_height)

The histogram + KDE curve helps us observe:  
- The central height tendency (mean around ~182 cm)  
- Spread of heights  
- Possible outliers  
- Whether the distribution is normal or skewed

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10,6))
plt.title("Player Height Distribution")
sns.histplot(players_unique["height"], kde=True, bins=30)
plt.xlabel("Height (cm)")
plt.ylabel("Count")
plt.grid(True, alpha=0.3)
plt.show()

### Insight
The average tennis player height is around **182 cm**.  
The distribution appears slightly right-skewed, meaning a small number of players are significantly taller than the average.

Most players fall between **175–190 cm**, which aligns with typical professional tennis standards.

### Question 10 — Correlation Between Player Height and Ranking
Interpretation

We want to check whether taller players tend to have higher or lower rankings.
We combine home & away players, remove duplicates, clean invalid values, and compute Pearson correlation.

In [ ]:
# Paths
base = "/Users/macbook/Downloads/Daneshkar/tennis project/TennisProject/data/processed/clean"

# Load datasets
df_home = pd.read_csv(os.path.join(base, "home_team_final.csv"))
df_away = pd.read_csv(os.path.join(base, "away_team_final.csv"))

# Merge home + away
players = pd.concat([df_home, df_away], ignore_index=True)

# Keep unique players
players_unique = players.drop_duplicates(subset="player_id")

# Replace invalid zeros with NaN
players_unique.loc[:, "height"] = players_unique["height"].replace(0, np.nan)
players_unique.loc[:, "current_rank"] = players_unique["current_rank"].replace(0, np.nan)

# Drop rows with missing required values
clean_players = players_unique.dropna(subset=["height", "current_rank"])

# Compute correlation
correlation = clean_players["height"].corr(clean_players["current_rank"], method="pearson")

print("correlation =" , correlation)


Correlation = 0.10355

Conclusion:

There is no meaningful correlation between player height and ranking.
Height does not significantly influence global ranking.

In [ ]:
plt.figure(figsize=(10,6))
sns.regplot(
    x="height",
    y="current_rank",
    data=clean_players,
    scatter_kws={"alpha":0.4},
    line_kws={"color":"red"}
)

plt.title("Height vs Ranking")
plt.xlabel("Height (cm)")
plt.ylabel("Ranking (Lower is Better)")
plt.grid(True, alpha=0.3)
plt.show()

###  Insight
The correlation coefficient was approximately **0.10**, indicating a **very weak positive correlation**.

This means **taller players tend to have slightly worse rankings**, but the effect is extremely small.  

Height does NOT strongly predict performance or ranking in professional tennis.

--------------------

### Question 11 — What is the average duration of matches?

To calculate the average match duration, we used the `time_final.csv` dataset, which contains
the duration of each period inside a tennis match:

- `period_1`
- `period_2`
- `period_3`

These periods contain the **duration in seconds**.

#### **Steps**
1. Replace NaN values in period columns with 0.  
2. Compute total match duration:  `duration_seconds = period_1 + period_2 + period_3`  
3. Keep only matches where duration > 0.  
4. Compute the mean duration.


In [ ]:
base = "/Users/macbook/Downloads/Daneshkar/tennis project/TennisProject/data/processed/clean"

df_time = pd.read_csv(os.path.join(base, "time_final.csv"))

# Replace NaN periods with 0
for col in ["period_1", "period_2", "period_3"]:
    df_time[col] = df_time[col].fillna(0)

# Compute duration in seconds
df_time.loc[:, "duration_seconds"] = df_time["period_1"] + df_time["period_2"] + df_time["period_3"]

# Filter out zeros (invalid matches)
df_valid = df_time[df_time["duration_seconds"] > 0].copy()
df_valid["duration_minutes"] = df_valid["duration_seconds"] / 60

# Average duration
avg_sec = df_valid["duration_seconds"].mean()
avg_minutes = avg_sec / 60
avg_hours = avg_minutes / 60

print("Average duration (seconds)=", avg_sec)
print("Average duration (minutes)=", avg_minutes)
print("Average duration (hours)=", avg_hours)


### **Final Answer**
- **Average duration (seconds):** 6705.87  
- **Average duration (minutes):** 111.76  
- **Average duration (hours):** 1.86  

#### **Average tennis match duration ≈ 1 hour and 52 minutes**

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Distribution of Match Duration")
sns.histplot(df_valid["duration_minutes"], kde=True, bins=40)
plt.xlabel("Duration (minutes)")
plt.ylabel("Count")
plt.grid(True, alpha=0.3)
plt.show()

###  Insight

The distribution shows:  
- Most matches are between **60 and 130 minutes**  
- A small number of matches last more than **180 minutes**  
- Very short or extremely long matches are rare  
  
  --------------------

### Question 12 - What is the average number of games per set in men's matches compared to women's matches?

To calculate the average number of games per set for men's and women's matches, we used three main datasets: `home_team_final.csv`, `away_team_final.csv`, and `pbp_final.csv`.

The `gender` column in the `home_team_final.csv` and `away_team_final.csv` datasets contains information about the **gender** of the players, and in the `pbp_final.csv` dataset we can also have `information about each game` to count it into women's and men's matches.

#### **Steps**
1. Reading datasets and storing them in code as dataframes
2. Create a `gender` dataframe that contains information about the gender and matches played by home and away players.
3. Obtaining the gender of each match using player grouping and getting the mode of the players for each match with the code `df_gender = df_gender.groupby("match_id")['gender'].apply(lambda x: x.mode()[0]).reset_index()`
4. Counting games per match and storing them in `df_games`
5. Merging the `df_gender` and `df_games` dataframes for the final calculation
6. Obtain the final result by grouping `df_merged` by gender and averaging for each gender (ignoring data where the `gender` column is unknown)
7. Obtaining the ratio of gender-unknown data to total data

In [ ]:
df_home = pd.read_csv(os.path.join(etl_files_path, "home_team_final.csv"))
df_away = pd.read_csv(os.path.join(etl_files_path, "away_team_final.csv"))
df_pbp = pd.read_csv(os.path.join(etl_files_path, "pbp_final.csv"))

df_gender = pd.concat(
    [df_home[['match_id', 'gender']],
     df_away[['match_id', 'gender']]],
    ignore_index=True)

df_gender = df_gender.groupby("match_id")['gender'].apply(lambda x: x.mode()[0]).reset_index()

df_games = df_pbp.groupby(['match_id', 'set_id',])['game_id'].nunique().reset_index(name='games_in_set')

df_merged = df_games.merge(df_gender, on='match_id', how='inner')

result = df_merged.groupby('gender')['games_in_set'].mean().drop('Unknown')

stat = df_merged.groupby('gender')['games_in_set']

ratio_of_unknown = (df_merged['gender'] == 'Unknown').sum() / len(df_merged) * 100

print(result)
print(ratio_of_unknown)
print('minimum games in a match', stat.max())

### **Final Answer**
- The average number of games per set in men's matches is approximately equal to **9.27**
- The average number of games per set in women's matches is approximately equal to **8.90**
- The highest number of games in a match for both women and men was **13**.

#### The proportion of players whose gender was unknown to the total dataset is approximately equal to **5.1%**

In [ ]:
fig, ax = plt.subplots(figsize=(16, 7))
sns.barplot(
    y=['Male', 'Female'],
    x=result.values,
    ax=ax,
    color='skyblue',
    hue=result.index,
    edgecolor = 'gray',
    palette=['skyblue', 'lightpink'],
    alpha = 0.9
)
ax.set_title('Average Number of Games per Match by Gender', fontsize=16, loc='left')
ax.set_ylabel('Gender', fontsize=14, loc='bottom', color='gray')
ax.set_xlabel('Games per Match', fontsize=14, loc="left", color='gray')
ax.tick_params(axis='both', which='major', colors='gray', labelsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('gray')
ax.grid(axis='x', linestyle='--', alpha=0.7)
ax.set_xticks(np.arange(0, 10, 1))
ax.legend().set_visible(False)
plt.show()

### Insight

The statistics shows:
- The average number of games in a match between women and men is not much different **(around 0.3 games per match)**.
- About **5.1%** of players had their gender unknown in the dataset and were not included in this statistic.

--------------------

### Question 13 — What is the distribution of left-handed versus right-handed players?

To calculate the distribution of left-handed versus right-handed players, we use two datasets, `home_team_final.csv` and `away_team_final.csv`, which contain player information.

In both of these datasets, there is a column called `plays` that stores information about whether the player is left-handed or right-handed.

#### **Steps**
1. Read the dataset and store it in code as data frames, uniquely considering each player and extracting only the plays column.
2. Creating df_all, which is a dataframe containing information about all players.
3. Create a distribution dataframe that counts and stores data for left-handed or right-handed players or for players whose status is unknown.

In [ ]:
df_home = pd.read_csv(os.path.join(etl_files_path, "home_team_final.csv")).drop_duplicates(subset='player_id')['plays']
df_away = pd.read_csv(os.path.join(etl_files_path, "away_team_final.csv")).drop_duplicates(subset='player_id')['plays']

df_all = pd.concat([df_home, df_away], ignore_index=True)

df_clean = df_all[df_all != 'unknown']

distribution = df_all.value_counts(dropna=True)
print(distribution)

ratio_of_unknown = (df_all == 'unknown').sum() / len(df_all) * 100
print('Ratio of unknown plays:', ratio_of_unknown)

distribution = distribution.drop('unknown', errors='ignore')

ratio_of_right_handed_without_unknown = (df_clean == 'right-handed').sum() / len(df_clean) * 100
print('Ratio of right-handed players:', ratio_of_right_handed_without_unknown)

ratio_of_left_handed_without_unknown = (df_clean == 'left-handed').sum() / len(df_clean) * 100
print('Ratio of left-handed players:', ratio_of_left_handed_without_unknown)

### **Final Answer**
- Without taking into account uncertain data, approximately **88.54% of players are right-handed**
-  and **11.45% of them are left-handed**.

#### Unfortunately, about **55.94%** of all players have data regarding their right- or left-handedness unknown.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 7))

sns.barplot(
    ax=ax,
    y=distribution.index,
    x=distribution.values,
    palette=['skyblue', 'lightgreen'],
    hue=distribution.index,
    alpha=0.9,
    edgecolor = 'gray'
    )
ax.set_title('Distribution of left-handed versus right-handed players', fontsize=16, loc='left')
ax.set_ylabel('Handedness', fontsize=14, loc='bottom', color='gray')
ax.set_xlabel('Number of Players', fontsize=14, loc="left", color='gray')
ax.tick_params(axis='both', which='major', colors='gray', labelsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('gray')
ax.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

### Insight

The statistics shows:
- Of all players, the left-handed or right-handed status of **2,749** is unknown, with **1,917** being right-handed and **248** being left-handed.
- The total number of players considered in this section was **4914**

--------------------

### Question 14 — What is the most common type of surface used in tournaments?

To calculate the most common field type in tournaments, we only need the `tournament_final.csv` dataset.

In this dataset, there is a column called `ground_type` through which you can find out the type of ground on which the matches were held.

#### **Steps**
1. Reading a dataset and storing it in a dataframe
2. Counting `ground_type`s in the data

In [ ]:
df_tournaments = pd.read_csv(os.path.join(etl_files_path, "tournament_final.csv"))

tournament_groundtype_counts = df_tournaments['ground_type'].value_counts().sort_values(ascending=False)

print(f'The most common ground type is: {tournament_groundtype_counts.idxmax()} with {tournament_groundtype_counts.max()} tournaments.')

ratio_of_unknown = (df_tournaments['ground_type'] == 'Unknown').sum() / len(df_tournaments) * 100
print('Ratio of unknown ground types:', ratio_of_unknown)
print(tournament_groundtype_counts)

### **Final Answer**
- The most common ground type is: Hardcourt outdoor
- Hardcourt outdoor has been used as a playing surface in 8116 tournaments

#### Only 1.61% of the data had an unknown playing surface.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 7))
sns.barplot(
    ax=ax,
    y=tournament_groundtype_counts.index,
    x=tournament_groundtype_counts.values,
    palette=['skyblue', 'gray', 'gray', 'gray', 'gray', 'gray', 'gray', 'gray', 'gray'],
    hue=tournament_groundtype_counts.index,
    edgecolor='gray',
    alpha = 0.9
    )
ax.set_title('Most common playing surfaces', fontsize=16, loc='left')
ax.set_ylabel('Playing Surface', fontsize=14, loc='bottom', color='gray')
ax.set_xlabel('Number of Tournaments', fontsize=14, loc="left", color='gray')
ax.tick_params(axis='both', which='major', colors='gray', labelsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('gray')
ax.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

### Insight

The statistics shows:
- There are about **16,873** tournaments in the dataset, with the most common playing surface among them being **Hardcourt outdoor**, which was present as the playing surface in **8,116** tournaments.
- The least popular playing surface among tournaments is **Green clay**, which has only been used as a playing surface in **18** tournaments.
--------------------

### Question 15 — How many distinct countries are represented in the dataset?

To calculate the individual countries in the dataset, we need to refer back to the `home_team_final.csv` and `away_team_final`.csv datasets.

In both of these datasets, there is a column called `country` for each player, which specifies the player's nationality and shows how many distinct countries there are in our data.

#### **Steps**
1. Reading datasets and storing only their `country` column in dataframes
2. Creating `df_country` which is a merged version of home and away countries
3. Counting distinct countries in `df_country`

In [ ]:
df_home = pd.read_csv(os.path.join(etl_files_path, "home_team_final.csv"))["country"]
df_away = pd.read_csv(os.path.join(etl_files_path, "away_team_final.csv"))["country"]

df_country = pd.concat([df_home, df_away], ignore_index=True)

country_counts = df_country.value_counts().sort_values(ascending=False)

country_counts_for_display = country_counts.head(10).drop(labels=["Unknown"], errors='ignore')

ratio_of_unknown = (df_country == 'Unknown').sum() / len(df_country) * 100
print('Ratio of unknown countries:', ratio_of_unknown)
print(country_counts.info())
print(country_counts.info())

### **Final Answer**
- There are **101 distinct countries** in our dataset, excluding unknown data.
- **France** has the highest number of players in our dataset.

#### About 29.72% of the data related to countries is unknown.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 7))
sns.barplot(ax=ax,
            x=country_counts_for_display.index,
            y=country_counts_for_display.values,
            hue=country_counts_for_display.index,
            edgecolor='gray',
            palette=['skyblue', 'skyblue', 'skyblue', 'skyblue', 'skyblue', 'skyblue', 'skyblue', 'skyblue', 'skyblue'],
            alpha=0.9)
ax.set_title(f'Top 10 Countries of Teams\nRatio of "Unknown" countries: {ratio_of_unknown:.2f}%\nThere are 101 distinct countries in the dataset.', loc='left', fontsize=16, color='gray', pad=20)
ax.set_xlabel('Country', loc='left')
ax.set_ylabel('Number of Teams', loc='bottom')
ax.tick_params(axis='both', colors='gray', labelsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis='y', linestyle='--', alpha=0.7)
sns.despine(ax=ax)
plt.show()

### Insight

The statistics shows:
- Of the 101 distinct countries, **France** had the highest number of players in the dataset.
- **Qatar**, **Bahamas** and **Haiti**, which are at the bottom of the table, each have only **1** player in the dataset.

--------------------

### Question 16 — Which player has the highest winning percentage against top 10 ranked opponents?

To find the player with the highest win rate against the 10 highest ranked players in the dataset, we need to use the datasets `home_team_final.csv`, `away_team_final.csv`, and `event_final.csv`.

To access player information, we use the home and away datasets, and to access match information, especially match winner information, we use the `winner_code` column event dataset.

#### **Steps**
1. Reading datasets and saving them in dataframes
2. Creating a `players` Table by Merging Home and Away
3. Extracting the top 10 players based on rank using the `current_rank` column
4. Extract all matches played by the top 10 players
5. Extracting top 10 competitors in matches alongside themselves
6. Merge the result with the event table to have the winner_code and the desired players in one place.
7. Determine if each player won their match
8. Filter NON–Top10 players only (the opponents)
9. Calculate win percentage vs Top10
10. Find the best player (sort by win rate and number pf matchs)

In [ ]:
df_home  = pd.read_csv(os.path.join(etl_files_path, "home_team_final.csv"))
df_away  = pd.read_csv(os.path.join(etl_files_path, "away_team_final.csv"))
df_event = pd.read_csv(os.path.join(etl_files_path, "event_final.csv"))

players = pd.concat([df_home, df_away], ignore_index=True)

# keep valid players only (remove rows with Unknown or rank=0)
players = players[(players["player_id"] != "Unknown") & (players["current_rank"] > 0)].copy()

# Create 'side' column: home=1, away=2
home_ids = df_home["player_id"].tolist()
players["side"] = players["player_id"].apply(lambda x: 1 if x in home_ids else 2)

players_unique = players.drop_duplicates(subset="player_id")
top10 = players_unique.sort_values("current_rank", ascending=False).head(10)

top10_ids = top10["player_id"].tolist()

matches_with_top10 = players[players["player_id"].isin(top10_ids)]["match_id"].unique()

subset_matches = players[players["match_id"].isin(matches_with_top10)].copy()

subset_matches = subset_matches.merge(
    df_event[["match_id", "winner_code"]],
    on="match_id",
    how="left"
)

subset_matches["is_winner"] = (
    ((subset_matches["side"] == 1) & (subset_matches["winner_code"] == 1)) |
    ((subset_matches["side"] == 2) & (subset_matches["winner_code"] == 2))
)

opponents = subset_matches[~subset_matches["player_id"].isin(top10_ids)].copy()

win_rate = opponents.groupby("player_id").agg(
    matches_vs_top10=("match_id", "count"),
    wins_vs_top10=("is_winner", "sum")
)

win_rate["win_percentage_vs_top10"] = (
    win_rate["wins_vs_top10"] / win_rate["matches_vs_top10"]
) * 100

# Add minimum matches threshold (recommended >= 5)
threshold = 5

win_rate_filtered = win_rate[win_rate["matches_vs_top10"] >= threshold]

# If no player passes threshold, fall back to >=2
if len(win_rate_filtered) == 0:
    win_rate_filtered = win_rate[win_rate["matches_vs_top10"] >= 2]

# Filter players that played minimum 2 matches vs top10
win_rate_filtered = win_rate[win_rate["matches_vs_top10"] >= 2]

# Sort by win rate first, then number of matches
best_player = win_rate_filtered.sort_values(
    ["win_percentage_vs_top10", "matches_vs_top10"],
    ascending=[False, False]
).head(1)

players_unique = players_unique[["player_id", "full_name", "current_rank"]]

best_player_named = best_player.reset_index().merge(
    players_unique,
    on="player_id",
    how="left"
)

print(f"Best player against top 10: {best_player_named['full_name'].values[0]} with ID: {best_player_named['player_id'].values[0]}")
print("he have played", best_player_named['matches_vs_top10'].values[0], "matches against top 10 players")
print(f"And he won {best_player_named['wins_vs_top10'].values[0]} of them.")
print(f"Win rate against top 10: {best_player_named['win_percentage_vs_top10'].values[0]:.2f}%")
print('the number of opponents', len(opponents.drop_duplicates(subset="player_id")))
print("the number of winning opponents", len(win_rate.drop_duplicates()))
print(f'player info:\n', players[players["player_id"] == best_player_named['player_id'].values[0]].drop_duplicates(subset="player_id"))


### **Final Answer**
- Best player against top 10: **Rocha, Francisco with ID: 228155**
- He have played 2 matches against top 10 players and She won 1 of them.
- Win rate against top 10: 50.00% with highest matchs number against top 10

#### This statistic does not have a plot.

### Insight

The statistics shows:
- There are **40** players as competitors for the top 10 ranked players.
- Among the **40** competitors, only **3** were able to win the game against the 10 highest ranked players.
- And the best of them is undoubtedly **Rocha, Francisco with ID: 228155**
- He is from **Portugal**, **185 cm tall**, weight unknown, current rank **934**.

--------------------

### Question 17 — What is the average number of breaks of serve per match?

To calculate the average break of serve per match, we only need the data set `power_final.csv`.

In this data, each row is for a game, there is a column called `break_occurred` that indicates whether a break occurred in that game.

#### **Steps**
1. Reading a dataset and saving it to a dataframe
2. Extract `break_occurred`s by grouping based on `match_id`, `set_num`, and `game_num` to get the breaks for each game, and with the help of the `any` function, we can understand the result of having a break or not when dealing with duplicate data.
3. Grouping breaks per game based on `match_id` and summing breaks per match
4. Averaging games per match
5. Filtering outlier data

In [ ]:
df_power = pd.read_csv(os.path.join(etl_files_path, "power_final.csv"))

breaks_per_game = df_power.groupby(['match_id', 'set_num', 'game_num'])['break_occurred'].any()

breaks_per_game = breaks_per_game.groupby('match_id').sum()

total_avg = breaks_per_game.mean()

breaks_per_game = breaks_per_game[(breaks_per_game <= 20) & (breaks_per_game > 0)]


print("Average breaks per match (excluding outliers):", total_avg)
print("Max breaks in a match (for filtered data):", breaks_per_game.max())
print("Total matches considered:", len(breaks_per_game))


### **Final Answer**
- The average number of breaks per match is **7.59**

#### This dataset does not have any NaN data.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 7))
sns.histplot(
            x=breaks_per_game,
            bins=20,
            kde=True,
            ax=ax,
            color='skyblue',
            label='Breaks per Match',
            edgecolor='gray',
            alpha=0.7)

ax.set_title('Distribution of Breaks per Match', loc='left', fontsize=16, color='black', pad=20)
ax.set_xlabel('Number of Breaks', loc='left')
ax.set_ylabel('Number of Matches', loc='bottom')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis='y', linestyle='--', alpha=0.7)
ax.legend(loc='upper left')
ax.tick_params(axis='both', colors='gray', labelsize=10)
ax.set_xticks(range(0, 21))
ax.axvline(total_avg, color='red', linestyle='--', label='Mean Breaks per Match')
ax.text(total_avg, ax.get_ylim()[1]*0.9, f'Mean: {total_avg:.2f}',
        rotation=90, color='red', verticalalignment='top', horizontalalignment='right')
plt.show()

### Insight

The statistics shows:
- About 10,968 matches were analyzed in this statistic, with the highest number of breaks per match being 20 breaks per match.
- A very small number of matches had 20 breaks per match, which were removed from the statistics as outlier data, and the data was kept up to 20 for the sake of the beauty of the distribution.

--------------------